<h1 style = "font-size:2rem;color:DarkCyan">Predicting feedback gain of FDN from impulse responses</h1>

In [1]:
#import libraries
import numpy as np
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import librosa, librosa.display 
import IPython.display as ipd
import scipy
import os
import pandas as pd

import sklearn

In [6]:
# import the dataset (extracted features) saved in a .csv
dataset = pd.read_csv(r'dataset.csv' , index_col=0)
dataset

,0,1,2,3,4,5,6,7,8,9,...,220491,220492,220493,220494,220495,220496,220497,220498,220499,feedback_gain
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-2.504054e-78,0.0,-1.666971e-81,0.0,1.292518e-63,0.0,-1.070469e-66,0.0,9.718300e-66,0.107655
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.424164e-21,0.0,-2.306065e-22,0.0,1.727031e-18,0.0,-3.932150e-19,0.0,8.767823e-19,0.438372
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-8.327647e-69,0.0,-1.423098e-71,0.0,4.874313e-56,0.0,-1.036550e-58,0.0,7.433983e-58,0.136276
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-5.405517e-31,0.0,-3.752988e-32,0.0,5.810936e-26,0.0,-5.219997e-27,0.0,1.468470e-26,0.347420
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-6.975886e-16,0.0,-1.760520e-16,0.0,5.411142e-14,0.0,-2.150335e-14,0.0,4.183179e-14,0.504200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-2.529944e-05,0.0,-1.067829e-05,0.0,5.843455e-06,0.0,1.209731e-05,0.0,2.099810e-05,0.652237
96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-9.133608e-51,0.0,-9.287349e-53,0.0,1.100412e-41,0.0,-1.396704e-43,0.0,6.408669e-43,0.213003
97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-4.078011e-40,0.0,-1.179057e-41,0.0,3.258109e-33,0.0,-1.186940e-34,0.0,4.184212e-34,0.277246
98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-3.458414e-79,0.0,-2.114401e-82,0.0,2.675004e-64,0.0,-2.034617e-67,0.0,1.886878e-66,0.105387


In [22]:
#impulse_response_stack = dataset[:,:44100 * 5]
feedback_gain_stack = np.array(dataset.feedback_gain)
feedback_gain_stack

0.1337080195840944

In [23]:
impulse_response_stack = np.array(dataset.iloc[:, :44100 * 5])
impulse_response_stack

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.07046871e-66,  0.00000000e+00,  9.71829968e-66],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -3.93215013e-19,  0.00000000e+00,  8.76782315e-19],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.03654958e-58,  0.00000000e+00,  7.43398308e-58],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.18694022e-34,  0.00000000e+00,  4.18421206e-34],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.03461738e-67,  0.00000000e+00,  1.88687800e-66],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -4.16720487e-45,  0.00000000e+00,  2.00015116e-44]])

In [39]:
librosa.feature.rms(y = impulse_response_stack[1]).shape

(1, 431)

In [40]:
def extract_features(data):
    return librosa.feature.rms(y = data)

sr = 48000

amount_of_features = 1

features = np.zeros((len(impulse_response_stack), 431))
target = feedback_gain_stack

idx = 0

for i in range(0, len(impulse_response_stack)):
    # extract features
    features[idx] = extract_features(impulse_response_stack[i])
    
    idx +=1
    

In [41]:
from sklearn.model_selection import train_test_split

#splitting the dataset in training and testing parts
feat_train, feat_test, tar_train, tar_test = train_test_split(features, target, test_size=0.3, random_state=5)


In [42]:
tar_train

array([0.28645663, 0.220827  , 0.26735669, 0.33707414, 0.10095212,
       0.20273827, 0.13165688, 0.57341491, 0.5418822 , 0.24636555,
       0.16101176, 0.38253884, 0.04320537, 0.1816875 , 0.29547971,
       0.61139661, 0.65223725, 0.34742048, 0.0996479 , 0.48477101,
       0.10538745, 0.66655754, 0.45038633, 0.26167676, 0.33632285,
       0.43778762, 0.49890636, 0.24527941, 0.44885615, 0.3694088 ,
       0.61622767, 0.67623217, 0.33510262, 0.13627572, 0.46410394,
       0.26889569, 0.17942507, 0.69768053, 0.50419988, 0.10765487,
       0.50934462, 0.52710872, 0.21300344, 0.43837194, 0.59415329,
       0.4950888 , 0.03056615, 0.35958915, 0.43365345, 0.10389527,
       0.10802946, 0.21727279, 0.26870243, 0.43839916, 0.16562328,
       0.19669731, 0.25064088, 0.10938193, 0.22748109, 0.22975984,
       0.29239829, 0.45281663, 0.5773758 , 0.0097057 , 0.25090502,
       0.47752197, 0.41506179, 0.50753591, 0.23405336, 0.20362543])

In [43]:
# Create linear regression object
regr = sklearn.linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(feat_train, tar_train)

# Make predictions using the testing set
tar_pred = regr.predict(feat_test)

In [44]:
#computing a set of performance metrics

#mean squared error (lower the better)
print('Mean squared error: %.4f'% sklearn.metrics.mean_squared_error(tar_test, tar_pred))

#mean absolute error (lower the better)
print('Mean absolute error: %.4f'% sklearn.metrics.mean_absolute_error(tar_test, tar_pred))

#mean absolute log error (lower the better)
print('Mean squared log error: %.4f'% sklearn.metrics.mean_squared_log_error(tar_test, tar_pred))

#maximum error (lower the better)
print('Max error squared error: %.4f'% sklearn.metrics.max_error(tar_test, tar_pred))

#median absolute error (lower the better)
print('Median absolute error: %.4f'% sklearn.metrics.median_absolute_error(tar_test, tar_pred))

#coefficient of determination (r2 score): 1 is perfect prediction (it can get arbitrary negative)
print('Coefficient of determination (R2 score): %.4f'% sklearn.metrics.r2_score(tar_test, tar_pred))

#explained variance score: 1 is perfect prediction (it can get arbitrary worse)
print('Explained variance score: %.4f'% sklearn.metrics.explained_variance_score(tar_test, tar_pred))


Mean squared error: 0.0000
Mean absolute error: 0.0000
Mean squared log error: 0.0000
Max error squared error: 0.0000
Median absolute error: 0.0000
Coefficient of determination (R2 score): 1.0000
Explained variance score: 1.0000


In [45]:
#displaying the individual errors
errors = np.absolute(tar_test - tar_pred)
print(errors)
print('errors')

#displaying names of poorly predicted files

errory_threshold = 0.01

#for index, value in errors.items():
#    if value > errory_threshold: #setting an arbitrary error threshold
#        print(filenames[index], value)

[9.38832345e-15 4.44089210e-16 3.33066907e-16 1.11022302e-16
 3.88578059e-16 4.30100400e-13 3.33066907e-16 8.88178420e-16
 3.33066907e-16 1.75836012e-11 1.11022302e-16 2.22044605e-16
 2.12049438e-08 3.05311332e-16 1.11022302e-15 3.05311332e-16
 8.32667268e-17 3.33066907e-16 3.33066907e-16 0.00000000e+00
 6.93889390e-17 5.55111512e-17 5.55111512e-17 1.11022302e-16
 0.00000000e+00 3.63647279e-08 1.11022302e-15 1.66533454e-16
 1.66533454e-16 1.11022302e-16]
errors


In [46]:
tar_test

array([0.039248  , 0.59135979, 0.36888694, 0.29745768, 0.12955101,
       0.65195257, 0.13370802, 0.48865974, 0.12999714, 0.65883495,
       0.27724591, 0.18077381, 0.68838943, 0.15645912, 0.56488073,
       0.15104283, 0.21853024, 0.51116663, 0.57900242, 0.43278007,
       0.10488427, 0.38161995, 0.25231102, 0.29143735, 0.4134444 ,
       0.69109568, 0.55436384, 0.36951844, 0.37996404, 0.3837481 ])

In [47]:
tar_pred

array([0.039248  , 0.59135979, 0.36888694, 0.29745768, 0.12955101,
       0.65195257, 0.13370802, 0.48865974, 0.12999714, 0.65883495,
       0.27724591, 0.18077381, 0.68838941, 0.15645912, 0.56488073,
       0.15104283, 0.21853024, 0.51116663, 0.57900242, 0.43278007,
       0.10488427, 0.38161995, 0.25231102, 0.29143735, 0.4134444 ,
       0.69109564, 0.55436384, 0.36951844, 0.37996404, 0.3837481 ])